In [1]:
import tmm_core as tmm_faller
import tmm
import pandas as pd
import numpy as np
import scipy as sp
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt

In [2]:
class Material:
    def __init__(self, path, name, wavelengths):
        f = pd.read_csv(path)
        wv_raw = np.array(f.wv)
        nc = np.array(f.n+f.k*1j)  # complex refractive index
        self.name = name
        self.f = interp1d(wv_raw, nc, kind='cubic')
        self.min_wv = min(wv_raw)
        self.max_wv = max(wv_raw)
        self.df = pd.DataFrame(nc, wv_raw, [name])
        # self.df = self.df.reindex(wavelengths)
        # self.df = self.df.interpolate('spline', order=3)
        self.interp(wavelengths)

    def interp(self, wavelengths):
        wavelengths = wavelengths[np.nonzero(np.logical_and(wavelengths > self.min_wv, wavelengths < self.max_wv))]
        nc = self.f(wavelengths)
        self.df = pd.DataFrame(nc,wavelengths, [self.name])

In [26]:
TCO = Material(".\Materials\Semiconductor\TCO.csv", "TCO", np.arange(300,1700))
Air = Material(".\Materials\Semiconductor\Air.csv", "Air", np.arange(300,1700))
CdSe = Material(".\Materials\Semiconductor\CdSe.csv", "CdSe", np.arange(300,1700))
CdTe = Material(".\Materials\Semiconductor\CdTe.csv", "CdTe", np.arange(300,1700))

materials = pd.DataFrame()
materials = materials.join([TCO.df, Air.df, CdSe.df, CdTe.df], how='outer')

#df.apply(tmm.coh_tmm, axis=1,args=(('p', [sp.inf,thickness,sp.inf], 0 , df.index)))
#thickness=0
#df.apply(tmm.list_snell,axis=1, args=(thickness,))
theta = 14
k = df.divide(np.array(df.index),axis=0)*2*sp.pi*sp.sin(theta*sp.pi/180)

In [35]:
df = materials[['Air', 'CdTe', 'TCO']]
ns = pd.Series(np.array(df).tolist(), index=df.index, name='n_list')
final = pd.DataFrame(index=df.index)
final=final.join(ns)

In [48]:
dd = {}
thetas = [0, 15, 30]
for theta in thetas:
    data = tmm_faller.coh_tmm('p', np.array(df) , [sp.inf,1000,sp.inf], theta*sp.pi/180 , np.array(df.index))
    final = pd.DataFrame(data=np.array([data['R'], data['T'], data['r'], data['t']]).T, index=data['lam_vac'], columns=['R', 'T', 'r', 't'])
    final = final.join(pd.Series(np.array(data['kz_list']).tolist(), index=df.index, name='kz_list'))
    dd[theta] = final
    
p = pd.Panel(dd)
p

<class 'pandas.core.panel.Panel'>
Dimensions: 3 (items) x 1389 (major_axis) x 5 (minor_axis)
Items axis: 0 to 30
Major_axis axis: 300 to 1688
Minor_axis axis: R to kz_list

In [94]:
import math
def alpha(n, lamb, s):
    return math.exp(-2*(s*n/lamb)**2)
    

d = 1000
s = 2*sp.pi*d

alpha(3,3500,s)


6.411600958072278e-26

In [ ]:
n_list = np.array(materials.loc[[1050], ['Air', 'CdTe', 'TCO']])[0]
df = pd.Series(name="R")
for thickness in range(1000):
    r = tmm.coh_tmm('p', n_list , [sp.inf,thickness,sp.inf],0 , 1050)['R']
    df = df.set_value(thickness, r)
plt.plot(df.index, df.values)
plt.xlabel("Thickness (nm)")
plt.ylabel("R")
plt.show()